<h1>Analysis of a specific matchup<h1>

0. Imports

In [73]:
import pandas as pd
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.static import teams
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import nba_analysis_helpers as methods
from plotly.subplots import make_subplots
from nba_api.stats.endpoints import leaguedashteamstats

<h2>Team parameters<h2>

In [74]:
team_1 = "Boston Celtics"
team_2 = "New York Knicks"

<h2>Make prepared logs for the teams<h2>

In [75]:
log_1 = methods.prepare_data(methods.get_team_log(team_1))
log_2 = methods.prepare_data(methods.get_team_log(team_2))


<h3>Here some plot about te teams ponts:<h3>

In [76]:
methods.make_hist_subplot(log_1,log_2,team_1,team_2,'PTS')

In [77]:
log_1['MATCHUP'] = methods.clean_matchup(log_1)
log_2['MATCHUP'] = methods.clean_matchup(log_2)

In [78]:
log_1_by_opponents = methods.scores_by_opponent(log_1, team_2)
log_2_by_opponents = methods.scores_by_opponent(log_2, team_1)

In [79]:
from IPython.display import display_html 


df1_styler = log_1_by_opponents.style.set_table_attributes("style='display:inline'").set_caption('df1')
df2_styler = log_2_by_opponents.style.set_table_attributes("style='display:inline'").set_caption('df2')
df2_t_styler = log_2_by_opponents.T.style.set_table_attributes("style='display:inline'").set_caption('df2_t')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)
# display_html(df1_styler._repr_html_()+df2_styler._repr_html_() +df2_t_styler._repr_html_(), raw=True)

,NYK
count,2.000000
min,114.000000
max,134.000000
mean,124.000000
median,124.000000
,BOS
count,2.000000
min,107.000000
max,138.000000
mean,122.500000


<h2>Lets see the season statics ot the two teams<h2>

In [80]:
team_1_ID = teams.find_teams_by_full_name(team_1)[0]['id']
team_2_ID = teams.find_teams_by_full_name(team_2)[0]['id']

teams_stats = pd.DataFrame(
    [leaguedashteamstats.LeagueDashTeamStats(team_id_nullable=team_1_ID).get_normalized_dict()['LeagueDashTeamStats'][0],
    leaguedashteamstats.LeagueDashTeamStats(team_id_nullable=team_2_ID).get_normalized_dict()['LeagueDashTeamStats'][0]]
    )

teams_stats[['TEAM_NAME', 'GP', 'W', 'L', 'W_PCT', 'FG_PCT', 'FG3_PCT','FT_PCT', 'PLUS_MINUS', 'CFID' ]]

,TEAM_NAME,GP,W,L,W_PCT,FG_PCT,FG3_PCT,FT_PCT,PLUS_MINUS,CFID
0,Boston Celtics,38,18,20,0.474,0.443,0.333,0.812,40.0,10
1,New York Knicks,38,18,20,0.474,0.436,0.353,0.774,-28.0,10


<h3>Lets visualize it:<h3>

In [81]:
methods.make_stat_comparison_subplot(teams_stats,team_1,team_2)

<h2>Lets make a train dataset to make a ML model to predict games<h2>

<h3>Table from the team stats by each game<h3>

In [82]:
from nba_api.stats.endpoints import teamgamelog
team_1_ID = teams.find_teams_by_full_name(team_1)[0]['id']
team_games = teamgamelog.TeamGameLog(team_id=team_1_ID).get_normalized_dict()["TeamGameLog"]

team_games = pd.DataFrame(team_games)
print(team_games.columns)
team_games.head()

Index(['Team_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'W', 'L', 'W_PCT',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS'],
      dtype='object')


,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612738,0022100570,"JAN 05, 2022",BOS vs. SAS,L,18,20,0.474,240,40,...,0.889,13,28,41,19,10,9,8,12,97
1,1610612738,0022100546,"JAN 02, 2022",BOS vs. ORL,W,18,19,0.486,265,39,...,0.893,9,37,46,25,5,10,21,23,116
2,1610612738,0022100529,"DEC 31, 2021",BOS vs. PHX,W,17,19,0.472,240,45,...,0.885,13,38,51,28,6,7,17,18,123
3,1610612738,0022100518,"DEC 29, 2021",BOS vs. LAC,L,16,19,0.457,240,35,...,0.727,21,29,50,19,8,8,8,16,82
4,1610612738,0022100503,"DEC 27, 2021",BOS @ MIN,L,16,18,0.471,240,33,...,0.750,15,38,53,23,8,10,14,16,103


In [83]:
#címke tömb
labels = team_games['WL']

#ebbe a táblába csak a statokat rakjuk
train_stats = team_games[['W', 'L', 'W_PCT']]

#hozzá kell adni egy plusz sort az elejére
train_stats = train_stats.append({'W':0,'L':0,'W_PCT':0}, ignore_index=True)
#meg kell fordítani a sorrendet
train_stats = train_stats[::-1]
#el kell dobni az utolsó sort
train_stats = train_stats.drop(0)
#indexeket a megfordítás miatt újra kell állítani
train_stats = train_stats.reset_index(drop=True)

#ide azokat amiket el kell csúsztatni
train = team_games[['GAME_DATE', 'MATCHUP']][::-1]
#indexeket a megfordítás miatt újra kell állítani
train = train.reset_index(drop=True)

#össze kell rakni a két elkészült táblákat 
train = pd.concat([train,train_stats], axis=1)

#át kell alakítani a matchupot úgy hogy értelmesebb, használhatóbb legyen
train['MATCHUP'] = train['MATCHUP'].map(lambda x: x.split(' ')[2])

train['GAME_DATE'] = pd.to_datetime(train['GAME_DATE'], format="%b %d, %Y")
train['day_in_month'] = train['GAME_DATE'].map(lambda x: int(x.strftime("%d")))
train['day_in_week'] = train['GAME_DATE'].map(lambda x: int(x.strftime("%w")))
train = train.drop('GAME_DATE', axis=1)

train.head()

,MATCHUP,W,L,W_PCT,day_in_month,day_in_week
0,NYK,0,0,0.000,20,3
1,TOR,0,1,0.000,22,5
2,HOU,0,2,0.000,24,0
3,CHA,1,2,0.333,25,1
4,WAS,2,2,0.500,27,3


<h3>Table from the team lineups<h3>

<h4>Some Preprocessing<h4>

In [84]:
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
encoder.fit(train.MATCHUP.unique())
encoder.classes_

array(['ATL', 'BKN', 'CHA', 'CHI', 'CLE', 'DAL', 'GSW', 'HOU', 'LAC',
       'LAL', 'MIA', 'MIL', 'MIN', 'NYK', 'OKC', 'ORL', 'PHI', 'PHX',
       'POR', 'SAS', 'TOR', 'UTA', 'WAS'], dtype=object)

In [85]:
train['MATCHUP'] = encoder.transform(train.MATCHUP)

In [103]:
train.head()

,MATCHUP,W,L,W_PCT,day_in_month,day_in_week
0,13,0,0,0.000,20,3
1,20,0,1,0.000,22,5
2,7,0,2,0.000,24,0
3,2,1,2,0.333,25,1
4,22,2,2,0.500,27,3


<h4>Split to train and test sets<h4>

In [87]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.3, random_state=12)

<h4>Make the model<h4>

In [88]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train,y_train)

model

GaussianNB()

<h4>Lets test on the test dataset<h4>

In [89]:
model.predict(X_test)

array(['L', 'W', 'W', 'L', 'L', 'L', 'W', 'W', 'L', 'W', 'W', 'W'],
      dtype='<U1')

In [126]:
model.score(X_test,y_test)

0.5833333333333334

In [127]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=4)
scores

array([0.57142857, 0.42857143, 0.5       , 0.5       ])

<h4>Now, we can predict the next game, firstly need to create a compatible table about the current stats<h4>

In [117]:
current_stats = team_games.iloc[0][['GAME_DATE','MATCHUP','W','L','W_PCT']]
current_stats['GAME_DATE'] = pd.to_datetime(current_stats['GAME_DATE'], format="%b %d, %Y")
current_stats['day_in_month'] = int(current_stats['GAME_DATE'].strftime("%d"))
current_stats['day_in_week'] = current_stats['GAME_DATE'].strftime("%w")
current_stats = current_stats.drop('GAME_DATE')

#TODO
current_stats['MATCHUP'] = 'BOS vs NYK'

matchup = encoder.transform([current_stats['MATCHUP'].split(' ')[2]])
w = [current_stats['W']]
l = [current_stats['L']]
wpct = [current_stats['W_PCT']]
month = [current_stats['day_in_month']]
week = [current_stats['day_in_week']]

current_stats = pd.DataFrame({'MATCHUP': matchup,'W': w, 'L': l, 'W_PCT': wpct, 'day_in_month':month, 'day_in_week': week})
current_stats

,MATCHUP,W,L,W_PCT,day_in_month,day_in_week
0,13,18,20,0.474,5,3


In [128]:
model.predict(current_stats)

array(['L'], dtype='<U1')